In [ ]:
# import samuroi and numpy, also select the proper matplotlib frontend.
%matplotlib qt4

import numpy
import samuroi

#### Load video data

In [ ]:
from samuroi.plugins.tif import load_tif

# load the multi image tif into a 3D numpy array 
data = load_tif('/path/to/your.tif')

# alternatively, one can directly load from a numpy file with `data=numpy.load(prefix+'.npy')`

#### Optional: image stabilization
The algorithm will try to remove stutters from the video with some optical flow calculations from opencv.

In [ ]:
try:
    # try import stabilization utility (requires opencv)
    from samuroi.plugins.stabilize import Stabilization 
    
    # the create stabilization tool and run it
    app = Stabilization(data = data)
    
    # correct the data by the stabilization transformations
    data = app.apply(data)        
except Exception as e:
    # print the reason why stabilization failed
    print e    

#### Create a morphology image from the video
A straight forward approach is to use a maximum projection of the video across the time axis.

In [ ]:
morphology = numpy.max(data, axis=-1)

#### Optional: Background Subtraction
Here some percentile values of the data are used to automatically determine a reasonable background value.

In [ ]:
background = numpy.percentile(data,q=5)
foreground = numpy.percentile(data,q=98)
print "background value: {}\n foreground value: {}".format(background,foreground)

# background subtraction
data = data - background

#### Optional: Frequency band filtering
Bandstop filtering the data is especially useful when data was acquired with a spinning disc confocal microscope. Here `fs` corresponds to sampling frequency in Hz, `start` and `stop` define the bandstop filtering window. 
The filter frequencies have to be manually read out from the plot of the power spectrum and the filtered data in cell below. In this example the frequencies from 18.95 Hz to 19.05 Hz are surpressed. The data is sampled on 20.83 Hz basis.

In [ ]:
# calculate power spectrum of data
df, psd = power_spectrum(data, fs = 20.83)

# filter some frequencies
data = bandstop(data, fs = 20.83 , start = 1,stop = 2)

# power spectrum of the unfiltered data
df, psdfiltered = power_spectrum(data, fs = 20.83)

# plot power spectra of unfiltered and filtered data to check the bandstop
import matplotlib.pyplot as plt

plt.figure()
plt.plot(df,psd, label = 'non-filtered')
plt.plot(df,psdfiltered, label = 'filtered')
plt.yscale('log')
plt.xlabel('frequency (Hz)')
plt.ylabel('Power Spectral Density')
plt.legend()

#### Optional: $\Delta F/F$ calculation

To approximate the base fluorescence value $F$, the data gets chunked into blocks of equal size, then the standard deviation for each block is calculated for each pixel. Then the mean value of the pixel intensity is calculated for the block with the smalest standard deviation and this value is taken as the base fluorescense value $F(x,y)$ where $x$ and $y$ are the pixel coordinates. 
Other options are (see `samuroi.plugins.baseline` package):
* linear bleech correction (`linbleeched_deltaF`)
* median correction (`median_deltaF`)

In [ ]:
from samuroi.plugins.baseline import stdv_deltaF

mean = numpy.mean(data, axis = -1)
data = stdv_deltaF(data)

#### Finally: Show the GUI

In [ ]:
app = samuroi.SamuROIWindow(data = data, morphology = morphology)

app.show()

#### Optional: Load an SWC file

Samuroi can either load files via the file dialog, or directly via ipython statements. Because the custom overlay will remove superfluos pixels from the mask, we can choose a larger radius for the loaded swc.

In [ ]:
from samuroi.plugins.swc import load_swc

swc = load_swc('/path/to/your.swc')

# increase radius of swc
swc.radius *= 2

# add the swc to the gui
app.segmentation.load_swc(swc)